The [NHS dictionary of medicines and devices (dm+d)](https://ebmdatalab.net/what-is-the-dmd-the-nhs-dictionary-of-medicines-and-devices/) has a flags to indicate as to whether a given item should be made available through hospital prescribing [(see tech spec of dm+d)](https://www.nhsbsa.nhs.uk/sites/default/files/2017-02/Data_Model_R2_v3.1_May_2015.pdf). We have [previously investigated some of the hospital flags](https://github.com/ebmdatalab/jupyter-notebooks/blob/3dc1f0d45173ab2dcf3156848df65e7a54be40f2/new_measures/Drafts/Hospital/Primary%20Care%20Prescribing%20of%20%20Hospital%20%20Only%20Medicines.ipynb). 

During our [work on zuclopenthixol](https://ebmdatalab.net/zuclopenthixol-acetate-a-new-kind-of-measure-on-openprescribing/) it became apparent that despite being a "hospital only" medicine, it was classified as such and is perhaps being administered inappropriately in primary care. 

Building on previous work this notebook sets out to explore primary care prescribing of "hospital only" medicines as defined by the [NHS dm+d](https://ebmdatalab.net/what-is-the-dmd-the-nhs-dictionary-of-medicines-and-devices/)


In [1]:
#import libraries required for analysis
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from ebmdatalab import bq
from ebmdatalab import charts
from ebmdatalab import maps

In [ ]:
sql = '''
WITH presentations_with_some_hospital_only_amps AS (
  SELECT bnf_code
  FROM dmd.amp
  WHERE avail_restrict = 8 and avail_restrict !=9 AND bnf_code IS NOT NULL ##8 relates to hospital only in dmd
  
  UNION DISTINCT 
  
  SELECT vmp.bnf_code
  FROM dmd.vmp
  INNER JOIN dmd.amp
    ON vmp.id = amp.vmp
  WHERE amp.avail_restrict = 8 AND vmp.bnf_code IS NOT NULL
)

SELECT
  pct,
  month,
  presentation AS bnf_name,
  bnf_code,
  SUM(items) AS total_items,
  SUM(actual_cost) AS total_cost
FROM
  hscic.normalised_prescribing_standard AS rx
INNER JOIN
  hscic.ccgs
ON
  rx.pct = ccgs.code
INNER JOIN
  hscic.bnf
ON
  rx.bnf_code = bnf.presentation_code
WHERE
  org_type = 'CCG'
  AND bnf_code IN (SELECT bnf_code FROM presentations_with_some_hospital_only_amps) 
GROUP BY
  pct,
  month,
  bnf_code,
  bnf_name
ORDER BY
  pct,
  month,
  bnf_code'''


df_hospital = bq.cached_read(sql, csv_path='data/df_hospital1.csv', use_cache=True)
df_hospital['month'] = df_hospital['month'].astype('datetime64[ns]')
df_hospital.head()

In [ ]:
df_hospital.nunique()

In [ ]:
df_hospital["bnf_name"].unique()

There are 290 unique names but at a glance they seem to be overwhelmingly what we would expect to be "hospital only" medicines. However many are for iv antibiotics and some other medicines where some areas have pioneered the use of specialist communiyt teams to adminster medicines in a patients own home to avoid a hospital admission. These may no longer be truly "hospital only".

In [ ]:
##groupby bnf name  to see "biggest offenders" in terms of items
df_product = df_hospital.groupby(['bnf_code', 'bnf_name']).sum().sort_values(by = 'total_items', ascending = False).reset_index()
df_product.head(25)

Some comments
- desogesterel is definitely not a hospital only medicine. It is a commonly used contraceptive. The [brand Moonia](https://openprescribing.net/dmd/amp/36546611000001102/) has been classed as hospital only and affects the figures.
- Clonazepam would not be considered a "hospital only" medicine. Reviewing dm+d it appears that a [product by Star Pharmaceuticals](https://openprescribing.net/dmd/advanced-search/amp/?search=%5B%22and%22%2C%5B%5B%22nm%22%2C%22contains%22%2C%22clonazep%22%5D%2C%5B%22avail_restrict%22%2C%22equal%22%2C%228%22%5D%5D%5D) has been designated "hospital only" while all other clonazepam products are not. I can't find the SPC for Star Pharmaceutical to see if there is some licnese difference but it is likely that this is a misclassification.
- Colistimeth Sod: This is used iv but also inhaled in treatment of lung infections particularly Cystic Fibrosis. [NHS England states](https://www.england.nhs.uk/wp-content/uploads/2019/04/nhs-england-drugs-list-v14.1.pdf) that when used for treatment of infections related to Cystic Fibrosis it is suiatable for shared care i.e. a GP could prescribe on FP10. 
Reviewing the [	Colistimethate 1million unit powder for solution for injection vials](https://openprescribing.net/dmd/vmp/324311004/)  AMPs more closely we find differences in the designations dependent on product. 

| AMP  | Hospital Only   |
|------|-----------------|
| [Colomycin](https://openprescribing.net/dmd/amp/3108611000001108/)*| No|
|[Promixin](https://openprescribing.net/dmd/amp/4658711000001106/) | Yes|
|[Kent Pharm](https://openprescribing.net/dmd/amp/17652211000001105/) | Yes|
|[AAH](https://openprescribing.net/dmd/amp/18758011000001102/)** | No |

        *The IV produc also has a license to be used by inhalation
        ** Most likely a make believe wholesaler product
    
- Antibiotics: The list is mainly antibiotics or palliatve care medicines. We do see these medicines in the community nowadays as the NHS seeks to keep people out of hospital. Potentially it is inapprorpriate to label these medicines "hospital only".

**Conclusion**
1 - We must investigate which products that are "mixed" i.e. should be excluded as it looks like an error in classification e.g. clonazepam and which should be retained for analysis as there is possibly a reason why AMPs differ e.g. colistin




In [ ]:
sql = '''WITH s AS (
  SELECT bnf_code, nm, avail_restrict
  FROM dmd.amp
  WHERE avail_restrict != 9 AND bnf_code IS NOT NULL

  UNION DISTINCT

  SELECT vmp.bnf_code, vmp.nm, amp.avail_restrict
  FROM dmd.vmp
  INNER JOIN dmd.amp
    ON vmp.id = amp.vmp
  WHERE amp.avail_restrict != 9 AND vmp.bnf_code IS NOT NULL

  ORDER BY bnf_code
)

SELECT bnf_code, nm,  hospital_only, COUNT(*) AS c
FROM (
  SELECT
    bnf_code,
    nm,
    CASE COUNT(CASE WHEN avail_restrict = 8 THEN 1 ELSE NULL END)
    WHEN 0 THEN "Not hospital only"
    WHEN COUNT(*) THEN "Hospital only"
    ELSE "Mixture"
    END AS hospital_only
  FROM
    s
  WHERE
    avail_restrict != 9 AND
    bnf_code IS NOT NULL    
  GROUP BY
    bnf_code, nm
  ORDER BY
    bnf_code
) WHERE hospital_only = "Mixture"
GROUP BY hospital_only, bnf_code, nm
ORDER BY bnf_code'''

df_mixed = bq.cached_read(sql, csv_path='data/df_mixed1.csv', use_cache=True)
pd.set_option('display.max_rows', None) 
df_mixed

On reviewing this list for the vast majority of products it makes no sense why one AMP would be "hospital only" and why another is not. These are probably inconsistencies as companies and the not NHS controls what is "hospital only". We should send this to BSA to support their review processes.

To continue our analyis let us manually filter the preparations that are obviously not (based on clinical experience) "hospital only" from df_product

In [ ]:
#'this is the list'
names = ["Clonazepam_Tab 500mcg", "Clonazepam_Tab 2mg", "Desogestrel_Tab 75mcg", "Tadalafil_Tab 20mg", "Clonazepam_Tab 2mg", "Ondansetron HCl_Tab 8mg"]



In [ ]:
## let us drop clonazepam before we proceed
#df_filtered = df_hospital[df_hospital["bnf_name"].str.contains(names) == False].sort_values("bnf_name")
df_filtered = df_hospital[~df_hospital['bnf_name'].isin(names)]
df_filtered.head()


In [ ]:
##here we plot trend over time.
df_filtered.groupby("month")['total_cost'].sum().plot(kind='line', title="Total cost of dm+d hospital only medicines prescribing")
plt.ylim(0, )

In 2016 there was a substanitial drop and now the costs are approximatelt £270k per month.

## Infrequently prescribed
Frequently prescribed items are probably not truly "hospital only" medicines anymore. We are interested in infrequently prescribed medicines that are "hospital only" and possibly a mistake. We highlighted some of this in our [zuclopenthixol blog](https://ebmdatalab.net/zuclopenthixol-acetate-a-new-kind-of-measure-on-openprescribing/) and Callum's [high dose dexamethaone blog](https://ebmdatalab.net/opendemocratisation-of-data-guest-blog-high-dose-dexamethasone/) where things aren't flagged as "hospital only" but probably are. Let us set the threshold of items in a year as similar (but arbitrary) to zuclopenthixol at 75 

In [ ]:
infreq = df_filtered.loc[(df_filtered['month'] >= '2018-11-01') & (df_filtered['month'] <= '2019-10-01')] ##restrict to last 12 months



In [ ]:
infreq2 = infreq.groupby(["bnf_name", "bnf_code"]).sum().sort_values('total_items', ascending=True).reset_index() ##group and sort by items
infreq2.to_csv("data/infreq_hosponly1.csv") ## export to csv for anyone for further investigation/emailing to interested folks


In [ ]:
## Now lets restict to 75 as per rationale above
inf_hosp_only = infreq2.loc[(infreq2['total_items'] < 75)]
pd.set_option('display.max_rows', None) 
inf_hosp_only

In the last year there are quite a few medicines that I have never heard of, that seem more suitable for secondary care but are being prescribed by practices. This is possible for rare unique cases although we must be mindful for errors abd this requires further investigation. For example is there any circumstance where a GP would prescribe _Rapifen Intensive Care_ which is a powerful opioid? Enbrel cost over £100k in primary care - this should be supplied through secondary care to avail of discounts? 

## Prototype Measure and Maps

In [ ]:
##get list size info for a measure per 1000 patients
sql2 = """
SELECT month, 
pct_id AS pct,
AVG(total_list_size) AS list_size
FROM ebmdatalab.hscic.practice_statistics
group by 
month, pct
order by
month, pct,
list_size
"""
df_list = bq.cached_read(sql2, csv_path='data/listsize1.csv')
df_list['month'] = df_list['month'].astype('datetime64[ns]')
df_list.head(5)

In [ ]:
ccg_total_hosp = df_filtered.groupby(["month", "pct"])["total_items","total_cost"].sum().reset_index()
ccg_total_hosp.head()

In [ ]:
x = df_filtered.groupby(["month", "pct", "bnf_name", "bnf_code"])["total_items","total_cost"].sum().reset_index()
x.head()

In [ ]:
#merge dataframes and cost / 10000

hosp_df_1000 = pd.merge(ccg_total_hosp, df_list, on=['month', 'pct'])
hosp_df_1000['cost_per_1000'] = 1000* (hosp_df_1000['total_cost']/hosp_df_1000['list_size'])
hosp_df_1000['items_per_1000'] = 1000* (hosp_df_1000['total_items']/hosp_df_1000['list_size'])
hosp_df_1000.head(5)

In [ ]:
#create sample deciles & prototype measure
charts.deciles_chart(
        hosp_df_1000,
        period_column='month',
        column='cost_per_1000',
        title="Hosp cost per 1000",
        show_outer_percentiles=False)

#add in example CCG (Islington)
df_subject = hosp_df_1000.loc[hosp_df_1000['pct'] == '08H']
plt.plot(df_subject['month'], df_subject['cost_per_1000'], 'r--')

plt.show()

In [ ]:
#create sample deciles & prototype measure
charts.deciles_chart(
        hosp_df_1000,
        period_column='month',
        column='items_per_1000',
        title="Hosp items per 1000",
        show_outer_percentiles=False)

#add in example CCG (Islington)
df_subject = hosp_df_1000.loc[hosp_df_1000['pct'] == '08H']
plt.plot(df_subject['month'], df_subject['items_per_1000'], 'r--')

plt.show()

This will require further thought and discussion with users before deploying a measure due to noise of  antibiotics etc. Potentially we could deploy a measure on the noise is excluded?

In [ ]:
#create choropeth map of cost per 1000 patients
plt.figure(figsize=(12, 7))
latest_hosp_df_1000 = hosp_df_1000.loc[(hosp_df_1000['month'] >= '2019-04-01') & (hosp_df_1000['month'] <= '2019-10-01')]
plt = maps.ccg_map(latest_hosp_df_1000, title="Hosp Cost per 1000 patients \n Apr - Oct 2019 ", column='cost_per_1000', separate_london=True)
plt.show()

In [ ]:
# CCGs may like to investigate further so lets produce a file for each of them.
# here we create a dataframe csv files for each CCG to investigate further if they wish - these are now available on GitHub

ccg_investigate = df_filtered.groupby(["month", "pct", "bnf_name", "bnf_code"])["total_items","total_cost"].sum().reset_index()


In [ ]:
#here we create the csv in a folder
    
    
        
for ccg_id, ccg_df in ccg_investigate.groupby('pct'):
     ccg_df.to_csv('ccg-data-for-review\{}.csv'.format(ccg_id), index=False)

## Electronic Health Records (EHR)

Our recent work has show prescribing has an effect on prescribing. Theroretically EHRs could be configured to prevent "hospital only" prescribing. Indeed they may be configured this way already. Here we investigate atterns as per EHR vendor.

In [ ]:
sql = """
SELECT
rx.month,
TRIM(Principal_Supplier) AS supplier,
rx.bnf_name,
rx.bnf_code,
SUM(items) AS total_items,
SUM(actual_cost) AS total_cost
FROM
 hscic.normalised_prescribing_standard AS rx
 JOIN (SELECT DISTINCT bnf_code 
  FROM ebmdatalab.dmd.amp
  WHERE avail_restrict = 8 AND bnf_code != '0408010F0AAABAB' AND ##clonazepam
bnf_code != '0408010F0AAACAC') AS dmd ##clonazepam
ON
  rx.bnf_code = dmd.bnf_code
JOIN
  hscic.ccgs AS ccgs
ON
  rx.pct=ccgs.code
JOIN
  alex.vendors AS software
ON
  software.ODS = rx.practice
  AND date(software.Date) = date(rx.month)
GROUP BY
rx.month,
supplier,
rx.bnf_name,
rx.bnf_code
ORDER BY
month"""



df_ehr_hosp = bq.cached_read(sql, csv_path='data/df_ehr_hosp1.csv')
df_ehr_hosp['month'] = df_ehr_hosp['month'].astype('datetime64[ns]')
df_ehr_hosp.head(3)

In [ ]:
sql = '''
WITH presentations_with_some_hospital_only_amps AS (
  SELECT bnf_code
  FROM dmd.amp
  WHERE avail_restrict = 8 and avail_restrict !=9 AND bnf_code IS NOT NULL ##8 relates to hospital only in dmd
  
  UNION DISTINCT 
  
  SELECT vmp.bnf_code
  FROM dmd.vmp
  INNER JOIN dmd.amp
    ON vmp.id = amp.vmp
  WHERE amp.avail_restrict = 8 AND vmp.bnf_code IS NOT NULL
)

SELECT
  TRIM(Principal_Supplier) AS supplier,
  month,
  presentation AS bnf_name,
  bnf_code,
  SUM(items) AS total_items,
  SUM(actual_cost) AS total_cost
FROM
  hscic.normalised_prescribing_standard AS rx
INNER JOIN
  hscic.ccgs
ON
  rx.pct = ccgs.code
INNER JOIN
  hscic.bnf
ON
  rx.bnf_code = bnf.presentation_code
INNER JOIN
  alex.vendors AS software
ON
  software.ODS = rx.practice
  AND date(software.Date) = date(rx.month)
WHERE
  org_type = 'CCG'
  AND bnf_code IN (SELECT bnf_code FROM presentations_with_some_hospital_only_amps) 
  AND bnf_code NOT LIKE "0703021Q0AA%" #desogestrel
  AND bnf_code NOT LIKE "0408010F0AA%" #clonazepam 
  AND bnf_code NOT LIKE "0704050R0AA%" #tadalafil
  AND bnf_code NOT LIKE "0406000S0AA%" #ondansetron
GROUP BY
  supplier,
  month,
  bnf_code,
  bnf_name
ORDER BY
  supplier,
  month,
  bnf_code
'''


df_ehr_hosp_only = bq.cached_read(sql, csv_path='data/df_ehr_hosp_only1.csv', use_cache=True)
df_ehr_hosp_only['month'] = df_ehr_hosp_only['month'].astype('datetime64[ns]')
df_ehr_hosp_only.head()

In [ ]:
##groupby bnf name  to see "biggest offenders" in terms of items
df_product_ehr = df_ehr_hosp_only.groupby(['bnf_code', 'bnf_name']).sum().sort_values(by = 'total_items', ascending = False)
df_product_ehr

In [ ]:
df_ehr_hosp_only.groupby(['month', 'supplier']).sum()['total_items'].unstack().plot.line()
plt.legend(loc=(1.04,0))
plt.title("total hosp only items per EHR vendor")

There is no striking trend and we only have EHR data since 2016. We should speak to EHR vendors, nerds and NHS Digital about this. We should have a look at user interface to support further work around EHR/dm+d.